In [1]:
from pyomo.environ import *
import os
models = []
for i in range(3):
    x = AbstractModel()
    models.append(x)
solverexe = "gurobi"
dirsolver = r"C:\Users\ch9fod\Documents\GitHub\ED\solvers"
datafile = "data1.dat"

In [2]:
#set
for i in range(3):
    models[i].G = Set()

In [3]:
#parameters
for i in range(3):
    models[i].a = Param(models[i].G)
    models[i].b = Param(models[i].G)
    models[i].Pmin = Param(models[i].G)
    models[i].Pmax = Param(models[i].G)
    models[i].d = Param(models[i].G)
    models[i].e = Param(models[i].G)
    models[i].f = Param(models[i].G)
    #lone parameter
    models[i].D = Param()

In [4]:
#variables
for i in range(3):
    models[i].P = Var(models[i].G)

In [5]:
#constraints
def maxp1(models,i):
    return models[1].P[i] <= models[1].Pmax[i]
models[1].maxprod = Constraint(models[1].G, rule = maxp1)

def maxp2(models,i):
    return models[2].P[i] <= models[2].Pmax[i]
models[2].maxprod = Constraint(models[2].G, rule = maxp2)

def minp1(models,i):
    return models[1].P[i] >= models[1].Pmin[i]
models[1].minprod = Constraint(models[1].G, rule = minp1)

def minp2(models,i):
    return models[2].P[i] >= models[2].Pmin[i]
models[2].minprod = Constraint(models[2].G, rule = minp2)

def demand_r0(models,i):
    return models[0].D == sum(models[0].P[i] for i in models[0].G)
models[0].demand = Constraint(models[0].G, rule = demand_r0)

def demand_r1(models,i):
    return models[1].D == sum(models[1].P[i] for i in models[1].G)
models[1].demand = Constraint(models[1].G, rule = demand_r1)

def demand_r2(models,i):
    return models[2].D == sum(models[2].P[i] for i in models[2].G)
models[2].demand = Constraint(models[2].G, rule = demand_r2)

In [6]:
#objective
def cost_rule0(models):
    return sum(models[0].a[i]*models[0].P[i] + 
               0.5*models[0].b[i]*models[0].P[i]**2 for i in models[0].G)
#default is to minimize        
models[0].OBJ = Objective(rule=cost_rule0) 

#objective
def cost_rule1(models):
    return sum(models[1].a[i]*models[1].P[i] + 
               0.5*models[1].b[i]*models[1].P[i]**2 for i in models[1].G)
#default is to minimize        
models[1].OBJ = Objective(rule=cost_rule1) 

#objective
def cost_rule2(models):
    return sum(models[2].a[i]*models[2].P[i] + 
               0.5*models[2].b[i]*models[2].P[i]**2 + 
               10*models[2].d[i] + 10*models[2].e[i]*models[2].P[i] + 
               10*models[2].f[i]*models[2].P[i]**2 for i in models[2].G)
#default is to minimize        
models[2].OBJ = Objective(rule=cost_rule2) 

In [8]:
solver = []
instance = []
results = []
for i in range(3):
    solver.append(SolverFactory(solverexe))
instance.append(models[1].create_instance(datafile))

# instance = model.create_instance(datafile)
# instance.dual = Suffix(direction=Suffix.IMPORT)
# results = solver.solve(instance)

ERROR: Rule failed when generating expression for constraint maxprod with index 1:
	ValueError: Error retrieving component unknown: The component has not been constructed.
ERROR: Constructing component 'maxprod' from data=None failed:
	ValueError: Error retrieving component unknown: The component has not been constructed.


ValueError: Error retrieving component unknown: The component has not been constructed.

In [ ]:
instance.display()

In [ ]:
p = [0, 0, 0 ,0]
for i in range(3):
    p[i+1] = value(instance.P[i+1])

In [ ]:
print ("Total Emissions (tonCO2/MWh)")
x = y = 0
for i in range(3):
    y = (instance.d[i+1] + instance.e[i+1]*p[i+1] + 
        instance.f[i+1]*p[i+1]**2)
    x = x + y
    print(i+1, y)
print("Total",x)

In [ ]:
print ("Duals")
from pyomo.core import Constraint
for c in instance.component_objects(Constraint, active=True):
    print ("   Constraint",c)
    cobject = getattr(instance, str(c))
    for index in cobject:
        print ("      ", index, instance.dual[cobject[index]])